In [1]:
import numpy as np
import pandas as pd 
import numpy.random as rn
from tqdm import tqdm_notebook as tqdm

from scipy import optimize       # to compare

/home/alice/Documents/ramax_vacation/venv/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
import pickle 

with open('possible_vacation.pickle', 'rb') as f:
    possible_vacation = pickle.load(f)

with open('req_work.pickle', 'rb') as f:
    req_work = pickle.load(f)
    
with open('excel_dict.pickle', 'rb') as f:
    excel_dict = pickle.load(f)

with open('qs.pickle', 'rb') as f:
    qs = pickle.load(f)

In [126]:
def annealing(random_start,
              cost_function,
              random_neighbour,
              acceptance,
              temperature,
              args,
              maxsteps=1000,
              debug=True
             ):
    state = random_start(*args)
    cost = cost_function(state, args[1])
    states, costs = [state], [cost]
    for step in range(maxsteps):
        fraction = step / float(maxsteps)
        T = temperature(fraction)
        new_state = random_neighbour(*args)  # fraction
        new_cost = cost_function(new_state, args[1])
#         if debug: print("Step #{:>2}/{:>2} : T = {:>4.3g}, state = {:>4.3g}, cost = {:>4.3g}, new_state = {:>4.3g}, new_cost = {:>4.3g} ...".format(step, maxsteps, T, state, cost, new_state, new_cost))
        if debug: print("Step #{:>2}/{:>2} : T = {:>4.3g} cost = {} new_cost = {} ...".format(step, maxsteps, T, cost, new_cost))          
        if acceptance_probability(cost, new_cost, T) > rn.random():
            state, cost = new_state, new_cost
            states.append(state)
            costs.append(cost)
    return state, cost_function(state), states, costs

In [127]:
def get_random_vac(max_fly_person, max_fly_month):
    return np.random.randint(36, min(max_fly_person, max_fly_month))


def random_matrix(df, q_name):
    state = np.zeros((df.shape[0], 12), dtype=int)
    max_fly_by_month = get_max_fly_by_month()
    aval_hours_by_month = get_aval_hours_by_month(df)

    for month in range(12):
        i = 0
        while aval_hours_by_month.loc[q_name, month] > 0:
            i += 1
            if state.shape[0] - i < 0:
                break
            rand_i = np.random.randint(0, state.shape[0])

            _vac = get_random_vac(
                max_fly_person=df['Max Fly'].iloc[rand_i],
                max_fly_month=max_fly_by_month[month]
            )

            if state[rand_i][month] > 0:
                continue

            # Если уже использовали больше чем нужно отпуск
            if state[rand_i].sum() + _vac > max(df['Max Fly'].iloc[rand_i] * 2, 184):
                continue

            # Если у нас на расстоянии 2 уже есть отпуск
            if month < 11:
                if state[rand_i][month + 1] > 0 or state[rand_i][month - 1] > 0:
                    continue
            else:
                if state[rand_i][0] > 0 or state[rand_i][10] > 0:
                    continue

            # Если число доступных часов больше
            if aval_hours_by_month.loc[q_name, month] >= _vac:
                aval_hours_by_month.loc[q_name, month] -= _vac
                state[rand_i, month] = _vac

            # Если число доступных часов меньше, то переносим отпуск на след месяц
            else:
                delta = _vac - aval_hours_by_month.loc[q_name, month]
                state[rand_i, month] = aval_hours_by_month.loc[q_name, month]
                aval_hours_by_month.loc[q_name, month] = 0

                if month < 11:
                    state[rand_i, month + 1] += delta
                    aval_hours_by_month.loc[q_name, month + 1] -= delta
                else:
                    state[rand_i, 0] += delta
                    aval_hours_by_month.loc[q_name, 0] -= delta
                break
    return state

In [128]:
possible_vacation

,0,1,2,3,4,5,6,7,8,9,10,11
Q1,8744.0,11250.0,6669.0,6941.0,11465.0,3163.0,3643.0,3940.0,4790.0,6266.0,9006.0,5103.0
Q2,6221.0,7151.0,6040.0,6178.0,6205.0,7021.0,6223.0,6253.0,6654.0,6519.0,7681.0,7196.0
Q3,2591.0,3521.0,2410.0,2548.0,2575.0,3391.0,2593.0,2623.0,3024.0,2889.0,4051.0,3566.0
Q4,767.0,3526.0,4067.0,3194.0,3361.0,1275.0,1813.0,1351.0,845.0,5295.0,2609.0,1897.0
Q5,873.0,3053.0,3386.0,3261.0,2637.0,1221.0,1742.0,1273.0,938.0,3288.0,2808.0,1980.0
Q6,9961.0,8308.0,9699.0,9504.0,11801.0,3802.0,5262.0,4606.0,4715.0,10796.0,6201.0,4953.0
Q7,1846.0,1686.0,1564.0,965.0,1323.0,840.0,777.0,1152.0,1309.0,694.0,1047.0,953.0
Q8,1036.0,1431.0,1331.0,998.0,1195.0,794.0,951.0,846.0,1000.0,1150.0,978.0,916.0
Q9,751.0,2913.0,1270.0,1768.0,2009.0,852.0,722.0,719.0,831.0,1766.0,1108.0,1114.0
Q10,4098.0,4753.0,4205.0,4702.0,3872.0,3601.0,3317.0,3317.0,3592.0,3361.0,3719.0,3446.0


In [129]:
def get_max_fly_by_month():
    return excel_dict['months'][2].values

def get_aval_hours_by_month(df):
    res = possible_vacation.copy()
    res.columns = range(12)
    return res

In [168]:
def get_D_d(vacs):
    """ Сколько часов удовлетворили """
    return vacs.sum(axis=0).sum()

def get_D(df, q_name):
    """ Запрос на часы отпуска """
    return possible_vacation.loc[q_name, :].sum()
    
def get_S(df):
    """ Суммарный рейтинг """
    return df[[f'RP{i}_score' for i in range(1, 13)]].values.sum().sum()
    
def get_S_d(vacs, df):
    """ Суммарный рейтинг выполненных заявок """
    a = df[[f'RP{i}_score' for i in range(1, 13)]].values
    b = vacs.astype(bool).astype(int)
    return np.multiply(a, b).sum().sum()

def get_N(df):
    """ Сколько заявок по отпуску всего"""
    return df[[f'RP{i}_score'  for i in range(1, 13)]].values.astype(bool).astype(int).sum().sum()

def get_N_d(vacs, df):
    """ Сколько заявок удовлетворили """
    req = df[[f'RP{i}_score'  for i in range(1, 13)]].values
    pos = vacs.astype(bool).astype(int)
    return np.multiply(req, pos).astype(bool).astype(int).sum().sum()

In [174]:
def cost_function(vacs, q_name):
    cost = []
    cost += [1 - get_S_d(vacs, df) / get_S(df)]
    cost += [1 - get_D_d(vacs) / get_D(df, q_name)]
    cost += [1 - get_N_d(vacs, df) / get_N(df)]
    return sum(cost) / len(cost)

In [175]:
def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        return 1
    else:
        p = np.exp(- (new_cost - cost) / temperature)
        return p

In [176]:
def temperature(fraction):
    return max(0.01, min(1, 1 - fraction))

In [177]:
def create_output(q_name, state, df):

    l = []

    for vac, person, person_score in zip(state, df['Num'].values, df[[f'RP{i}_score' for i in range(1, 13)]].values):
        for i, (vac_size, score) in enumerate(zip(vac, person_score), 1):
            if vac_size != 0:
                l.append([person, i, vac_size, 1 if score else 0])
    
    _df = pd.DataFrame(data=l, columns=['id', 'month', 'rest_size', 'application'])
    _df.to_csv(f'../data/res_v.0.03_{q_name}.csv')

In [178]:
df_costs = pd.DataFrame(data = [0] * possible_vacation.shape[0], index=possible_vacation.index)

for q_name, df in qs.items():
    state, c, states, costs = annealing(
        random_matrix,
        cost_function,
        random_matrix,
        acceptance_probability,
        temperature,
        args=[df, q_name],
        maxsteps=100,
        debug=True
    )
    df_costs.loc[q_name, 0] = 
    create_output(q_name, state, df)

Q1
Step # 0/100 : T =    1 cost = 0.5002511466084724 new_cost = 0.4949653752665544 ...
Step # 1/100 : T = 0.99 cost = 0.4949653752665544 new_cost = 0.490816199194355 ...
Step # 2/100 : T = 0.98 cost = 0.490816199194355 new_cost = 0.5113598419951558 ...
Step # 3/100 : T = 0.97 cost = 0.5113598419951558 new_cost = 0.49524827078542816 ...
Step # 4/100 : T = 0.96 cost = 0.49524827078542816 new_cost = 0.4997468211586693 ...
Step # 5/100 : T = 0.95 cost = 0.4997468211586693 new_cost = 0.5142254531060249 ...
Step # 6/100 : T = 0.94 cost = 0.5142254531060249 new_cost = 0.5051790130613055 ...
Step # 7/100 : T = 0.93 cost = 0.5051790130613055 new_cost = 0.5148029800311956 ...
Step # 8/100 : T = 0.92 cost = 0.5148029800311956 new_cost = 0.5012677907182722 ...
Step # 9/100 : T = 0.91 cost = 0.5012677907182722 new_cost = 0.5037349156307519 ...
Step #10/100 : T =  0.9 cost = 0.5037349156307519 new_cost = 0.5361909916710813 ...
Step #11/100 : T = 0.89 cost = 0.5361909916710813 new_cost = 0.4958089472

KeyboardInterrupt: 